In [11]:
import tkinter as tk
from tkinter import messagebox, ttk
import mysql.connector
from datetime import datetime
from tkcalendar import DateEntry


root = None
login_window = None
username_entry = None
password_entry = None
current_user_id = None
current_username = None


def connect_to_db():
    try:
        conn = mysql.connector.connect(
            host="localhost",
            user="root",
            password="sid@123",
            database="expense_trackerrr"
        )
        return conn
    except mysql.connector.Error as err:
        messagebox.showerror("Database Error", f"Error: {err}")
        return None


def validate_login(username, password):
    conn = connect_to_db()
    if conn:
        cursor = conn.cursor()
        try:
            cursor.execute("SELECT user_id, username FROM users WHERE username=%s AND password=%s", (username, password))
            user = cursor.fetchone()
            if user:
                return user
            else:
                return None
        except mysql.connector.Error as err:
            messagebox.showerror("Database Error", f"Error: {err}")
            return None
        finally:
            cursor.close()
            conn.close()


def on_login():
    global current_user_id, current_username
    username = username_entry.get()
    password = password_entry.get()
    user = validate_login(username, password)
    if user:
        current_user_id = user[0]
        current_username = user[1]
        login_window.destroy()
        show_expense_tracker()
    else:
        messagebox.showerror("Login Failed", "Invalid username or password")


def add_expense(date, category, description, amount):
    conn = connect_to_db()
    if conn:
        cursor = conn.cursor()
        try:
            global current_user_id
            if current_user_id is None:
                messagebox.showerror("User Error", "No user logged in.")
                return
            
            cursor.execute(
                "INSERT INTO expenses (user_id, date, category, description, amount) VALUES (%s, %s, %s, %s, %s)",
                (current_user_id, date, category, description, amount)
            )
            conn.commit()
            messagebox.showinfo("Success", "Expense added successfully!")
        except mysql.connector.Error as err:
            messagebox.showerror("Database Error", f"Error: {err}")
        finally:
            cursor.close()
            conn.close()


def view_expenses():
    conn = connect_to_db()
    if conn:
        cursor = conn.cursor()
        try:
            global current_user_id
            if current_user_id is None:
                messagebox.showerror("User Error", "No user logged in.")
                return
            
            cursor.execute("SELECT date, category, description, amount FROM expenses WHERE user_id = %s", (current_user_id,))
            rows = cursor.fetchall()
            for row in tree.get_children():
                tree.delete(row)
            for row in rows:
                date = row[0].strftime("%d %m %Y")  
                tree.insert("", "end", values=(date, row[1], row[2], row[3]))
        except mysql.connector.Error as err:
            messagebox.showerror("Database Error", f"Error: {err}")
        finally:
            cursor.close()
            conn.close()


def on_add_expense():
    date = date_entry.get_date().strftime("%Y-%m-%d")
    category = category_var.get()
    description = description_entry.get()
    amount = amount_entry.get()
    
    if not date or not category or not amount:
        messagebox.showwarning("Input Error", "Date, Category, and Amount are required fields")
        return

    try:
        amount = float(amount)
    except ValueError:
        messagebox.showwarning("Input Error", "Amount must be a number")
        return
    
    add_expense(date, category, description, amount)
    view_expenses()  
    date_entry.set_date(datetime.now())
    description_entry.delete(0, tk.END)
    amount_entry.delete(0, tk.END)


def show_expense_tracker():
    global root, date_entry, category_var, description_entry, amount_entry, tree

    root = tk.Tk()
    root.title(f"Expense Tracker - Logged in as {current_username}")
    root.configure(bg="skyblue")

    tk.Label(root, text="Date (DD MM YYYY)").grid(row=0, column=0, padx=10, pady=10)
    date_entry = DateEntry(root, date_pattern='dd mm yyyy')
    date_entry.grid(row=0, column=1, padx=10, pady=10)

    tk.Label(root, text="Category").grid(row=1, column=0, padx=10, pady=10)
    categories = ["Food", "Transportation", "Entertainment", "Utilities", "Other"]
    category_var = tk.StringVar(root)
    category_dropdown = ttk.Combobox(root, textvariable=category_var, values=categories)
    category_dropdown.grid(row=1, column=1, padx=10, pady=10)
    category_dropdown.current(0)

    tk.Label(root, text="Description").grid(row=2, column=0, padx=10, pady=10)
    description_entry = tk.Entry(root)
    description_entry.grid(row=2, column=1, padx=10, pady=10)

    tk.Label(root, text="Amount").grid(row=3, column=0, padx=10, pady=10)
    amount_entry = tk.Entry(root)
    amount_entry.grid(row=3, column=1, padx=10, pady=10)

    add_button = tk.Button(root, text="Add Expense", command=on_add_expense)
    add_button.grid(row=4, column=0, columnspan=2, pady=10)

    view_button = tk.Button(root, text="View Expenses", command=view_expenses)
    view_button.grid(row=5, column=0, columnspan=2, pady=10)

    columns = ("Date", "Category", "Description", "Amount")
    tree = ttk.Treeview(root, columns=columns, show="headings")
    tree.heading("Date", text="Date")
    tree.heading("Category", text="Category")
    tree.heading("Description", text="Description")
    tree.heading("Amount", text="Amount")
    tree.grid(row=6, column=0, columnspan=2, padx=10, pady=10)

    root.mainloop()


def show_login_window():
    global login_window, username_entry, password_entry

    login_window = tk.Tk()
    login_window.title("Login")

    tk.Label(login_window, text="Username").grid(row=0, column=0, padx=10, pady=10)
    username_entry = tk.Entry(login_window)
    username_entry.grid(row=0, column=1, padx=10, pady=10)

    tk.Label(login_window, text="Password").grid(row=1, column=0, padx=10, pady=10)
    password_entry = tk.Entry(login_window, show='*')
    password_entry.grid(row=1, column=1, padx=10, pady=10)

    login_button = tk.Button(login_window, text="Login", command=on_login)
    login_button.grid(row=2, column=0, columnspan=2, pady=10)

    login_window.mainloop()


def main():
    show_login_window()

if __name__ == "__main__":
    main()
